# Notes, 9/6/18

## Mac homebrew

### Build from source

* This can be used to make sure your libraries have been compiled using the same compilers that you are going to use for JEDI

* Need to do this on the mac since the default compilers are clang
    * These have access to gcc, but it's an ancient version (4.x)
    
* Add the --build-from-source option to the brew install command
    * --HEAD will check out a version from a repository
    * you can define "head" in the brew formula which gives you the ability to define a particular branch or tag to use

### Edit a formula

* Formulas are ruby scripts
    * To set an environment variable in a formula
        * ENV\['FC'\] = '/usr/local/bin/gfortran-7'

* brew edit <formula\>


* After finishing you can check your formula by "auditing"
    * brew audit <formula\>
    * This checks for acceptability to merge your formula, or edits to and existing formula into the brew repository

### Create your own formula

* Need source code
    * Brew wants source code so that it can be demonstrated that you can build on your platform

* Two ways to provide source code:
    * Tar file
        * Give brew path to a tarball containing the source code and build configuration
        * Can use autoconf, cmake, etc.
        * Give brew the sha256 sum for the tar file
            * shasum -a256 <tar_file\>
    * Repository
        * Give brew url for cloning
        * Can specify particular branches or tags
        * Give brew the sha1 value for the commit you are checking out

* Create a new formula
    * brew edit <formula\>
    
### Environment for homebrew

* export HOMEBREW_CC="gcc-7"
* export HOMEBREW_CXX="g++-7"

* brew --env
* brew config

### Building boost

* brew install --verbose --build-from-source --c++11 boost
   * says it is ignoring c++11, but it uses c++1 anyway
   * c++11 is necessary to get boost name mangling to sync up with JEDI name mangling

### Formulas for open-mpi, hdf5, netcdf

* brew edit open-mpi, hdf5, netcdf
    * Change depends_on "gcc" to "gcc@7"
    * For Fortran compiling, add: ENV\['FC'\] = '/usr/local/bin/gfortran:q'
    
* For open-mpi only
    * brew edit open-mpi
        * add to the head section
            * :using =\> :git,
            * :branch =\> "v3.2.1",
            * :revision =\> "<full sha value from git log\>"
        * ENV\['CC'\] = '/usr/local/bin/gcc-7'
        * ENV\['CXX'\] = '/usr/local/bin/g++-7'

* For netcdf only
    * do a brew edit to see the url for downloading netcdf-fortran source tarball
    * do a wget to download this tarball, unpack it, and edit the top level CMakeLists.txt
        * get rid of the macro-backtrace options to gfortran (these are for clang, not gcc)
    * repack the tar ball, and store in a safe place
    * run shasum -a256 on the tarball
    * brew edit netcdf
        * in resource "fortran" do
            * comment out the specs and replace with url and sha256 values in accordance with the new tarball
                * use the file://<full path\> form of the url

* brew install --verbose --build-from-source open-mpi, hdf5, netcdf
   * Add the --c++11 option for netcdf
   * Don't use the --HEAD option for open-mpi
        
### Formulas for ecbuild, eckit, fckit

* Need to create new formulas for these

* brew create ecbuild, eckit, fckit

* For ecbuild:
~~~~~~~~
class Ecbuild < Formula
  desc "CMake macros for building JEDI"
  homepage "https://github.com/ECMWF/"
  url "https://github.com/ECMWF/ecbuild.git",
    :using => :git,
    :tag => "2.9.0",
    :revision => "10d46b3d22df192405aa4da148e5c80bd7a814e0"

  depends_on "cmake" => :build

  def install
    mkdir "build" do
      system "cmake", "..", *std_cmake_args
      system "make", "install"
    end
  end

  test do
  end
end
~~~~~~~~

* For eckit:
~~~~~~~~
class Eckit < Formula
  desc "C++ utilities for JEDI"
  homepage "https://github.com/ECMWF"
  url "https://github.com/ECMWF/eckit.git",
    :using => :git,
    :tag => "0.22.0",
    :revision => "a1d8af4a48cdfc9088761a2ddf69eceb69e4ba13"

  depends_on "cmake" => :build
  depends_on "ecbuild"
  depends_on "eigen"
  depends_on "open-mpi"

  def install
    # Allow open-mpi to oversubscribe processes since
    # only have 2 cores on this system.
    ENV['OMPI_MCA_rmaps_base_oversubscribe'] = '1'
    mkdir "build" do
      system "ecbuild", "..", *std_cmake_args
      system "make"
      system "ctest"
      system "make", "install"
    end
  end

  test do
  end
end
~~~~~~~~

* For fckit:
~~~~~~~~
class Fckit < Formula
  desc "Fortran utilities for JEDI"
  homepage "https://github.com/ECMWF/"
  url "https://github.com/ECMWF/fckit.git",
    :using => :git,
    :tag => "0.5.2",
    :revision => "dad53e01cb6353476ca9c890071713dca353da10"

  depends_on "cmake" => :build
  depends_on "ecbuild"
  depends_on "eckit"

  def install
    # Allow open-mpi to oversubscribe processes since
    # only have 2 cores on this system.
    ENV['OMPI_MCA_rmaps_base_oversubscribe'] = '1'
    mkdir "build" do
      system "ecbuild", "..", *std_cmake_args
      system "make"
      system "ctest"
      system "make", "install"
    end
  end

  test do
  end
end
~~~~~~~~

* To install: brew install --verbose ecbuild, eckit, fckit



# MPICH2 on Mac, 7/30/19

* JEDI compiles okay using
    * Mojave
    * Clang 10.0.1
    * GNU 9.1.0
    * Mpich 3.3.1
* But get a lot of test failures
    * "channel initialization failed"
    * "sethostbyname failed, imac.fin.ucar.edu"
* Fix for this is to add an entry to the /etc/hosts file:

~~~~~~~~
hostname      # get the local host name
              # on my imac this returns "sysadmins-imac.fin.ucar.edu"

sudo vi /etc/hosts
  # add entry that uses the result of above as the second item on the line
  # For my imac this is:
  
  127.0.0.1 sysadmins-imac-fin.ucar.edu
  127.0.0.1 imac-fin.ucar.edu            # this works too
~~~~~~~~

* When running on my MacBook, using Mpich, I get the following error in addition to the ones I saw on the iMac
    * MacBook config:
        * High Sierra
        * Clang 10.0.0
        * GNU 7.4.0
        * Mpich2 3.3.1
* From OOPS, the test called test_util_intset_parser fails
    * for the test_get_channels_invalid section, the EXCEPT_THROWS() doesn't catch the exception and the program crashes
    * Don't have this dubugged yet
* Went back to openMPI on my MacBook

# Catalina OS, 11/8/19

* Fix file permissions for iTerm
    * Without this, ls won't work on ~/Documents and others
    * Set "Full Disk Access" for iTerm in the Security and Privacy system preferences
    * Instructions: http://osxdaily.com/2018/10/09/fix-operation-not-permitted-terminal-error-macos/
* Use brew to uninstall, then reinstall everything with Catalina compiled versions
    * Use Nan's list from his notes on mac build
* Go to "System Preferences" and install 11.0 and 11.2 versions of command line librarys
* Use jedi-stack build process to re-build with Clang 11.0
    * Got a link error during the build of eckit
        * Cannot link dylib directly, crypto library from openssl is out of date
        * Workaround:

```
brew upgrade openssl # openssl should already be there, but if not: brew install openssl

cd /usr/local/lib
ln -s ../Cellar/openssl/lib/libssl.1.0.0.dylib libssl.dylib
ln -s ../Cellar/openssl/lib/libcrypto.1.0.0.dylib libcrypto.dylib

# repeat the jedi-stack build process, eckit should complete this time
```

# Vagrant notes, 3/6/20

* Error: dpkg-reconfigure: unable to re-open stdin: No file or directory
    * From ubuntu VM
    * apt-get is attmpting to run in interactive mode, and is waiting for response from console
    * Fix:
        * export DEBIAN_FRONTEND=noninteractive

# Mac OS, System Integrity Protection (SIP), 5/4/20

* This is enabled by default

* Among other things, this strips DYLD_LIBRARY_PATH and LD_LIBRARY_PATH from environment
    * Big impact on python scripts
    * #!/usr/bin/env ptyhon
        * This uses python as the interpreter, but the library path variables are missing

* To shut off
    * Reboot and hold dowd Cmd-R
    * This goes into Recovery Mode
    * From Recovery Mode
        * Select Utilities -> Terminal
        * In the terminal, enter

```
csrutil status  # check if enabled, disabled
csrutil disable # shut of SIP
reboot
```

# Dynamic loader search order, 5/21/20

* Order is different depending on whether or not the file is specified with a path
    * Ie, in the call to dlopen()

* When just a file name (no path)
    1. LD_LIBRARY_PATH
    2. DYLD_LIBRARY_PATH
    3. process' working direcotry
    4. DYLD_FALLBACK_LIBRARY_PATH

* WHen path is included
    1. DYLD_LIBRARY_PATH
    2. Given path/filename
    3. DYLD_FALLBACK_LIBRARY_PATH

* Surprising that when the path is included, the first thing tried is DYLD_LIBRARY_PATH instead of the given path/filename

# Dynamic loader namespace, 5/21/20

* The Mac OS dynamic loader (dyld) by default uses a feature called two-level namespace
   * two-level namespace causes the link step to record where every reference in one library is resoloved in another
   * during run time, the dynamic loader follow the recording that the link step created
   * this allows both Clang and GNU std libraries to be linked to libraries containing C/C++/Fortran code and have the dynamic loader load the proper std library dylib files

* Using -flat_namespace flag during the link step disables two-level namespace (using a flat namespace instead)
   * This disables the recording of the link structure during the link step
   * The dynamic loader at runtime uses the typical LD_LIBARAY_PATH, ... search
   * This allows the Mac OS to emulate what most other Linux systems do (which can be useful)
   * However, this can cause issues with libraries that have a C/C++/Fortran code mix
       * Because the proper std library under each call is lost